<h1 style="text-align: center;">Exploratory Data Analysis of G2F-Indigo Metadata</h1>

 Write brief intro and major steps in data wrangling.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
sns.set_palette('Set2')

![alt text](corn_graphic.png "Corn Stages")

In [ ]:
data = pd.read_csv('G2F_Indigo_metadata.csv')

In [ ]:
data.shape

In [ ]:
data.head(10)

In [ ]:
data.dtypes

<h2 style="text-align: center;">Missing Values and Duplicates</h2>

In [ ]:
# Plot of missing values in each column, add value to bars

missing_values = data.isnull().sum()
plt.figure(figsize=(10, 6))
na_plot = plt.bar(missing_values.index, missing_values.values)
plt.bar_label(na_plot, label_type = 'edge')
plt.xticks(rotation = 90)
plt.xlabel('Feature')
plt.ylabel('Number of Missing Values')
plt.title('Missing Value Count by Feature')
plt.show()

In [ ]:
# Convert sample_id and plant_id to Int64 to deal with missing values
data['sample_id'] = data['sample_id'].astype('Int64')
data['plant_id'] = data['plant_id'].astype('Int64')

In [ ]:
# Zea mays the only species?
data['crop'].unique()

In [ ]:
# Duplicate rows? None

data[data.duplicated()]

In [ ]:
# Look at duplicates in sequence_label (This connects to FASTQ files), sort values

seq_label_dups = data[data.duplicated(subset=['sequence_label'], keep=False)]
seq_label_dups = seq_label_dups.sort_values(by='sequence_label')

seq_label_dups.head(20)

In [ ]:
print(f'Total duplicated sequence_label entries: {int(len(seq_label_dups) / 2)}')

In [ ]:
# Groupby sequence_label,  check if all True
grouped = seq_label_dups.groupby('sequence_label')['sample_id']

# check if sample_id had exactly one Nan and exactly one Int
pattern_check = grouped.apply(lambda x: x.isnull().sum() == 1 and x.dropna().nunique() == 1) # Boolean series

# Is Int entry index < NaN entry index?
index_check = grouped.apply(lambda x: list(x[~x.isnull()].index)[0] < list(x[x.isnull()].index)[0])

print(f'All same pattern? {pattern_check.all()}')

print(f'For all, Int index < NaN index? {index_check.all()}')

In [ ]:
print(f'Entries Before Drop: {len(data)}')

# Drop duplicates in sequence_label, keep Int entry in sample_id

data = data.drop_duplicates(subset=['sequence_label'], keep='first')

print(f'Entries After Drop: {len(data)}')

In [ ]:
# drop entry if plant_ id is missing
data = data.dropna(subset=['plant_id'])

In [ ]:
missing_values_2 = data.isnull().sum()
plt.figure(figsize=(10, 6))
plot2 = plt.bar(missing_values_2.index, missing_values_2.values)
plt.bar_label(plot2, label_type = 'edge')
plt.xticks(rotation = 90)
plt.xlabel('Feature')
plt.ylabel('Number of Missing Values')
plt.title('Missing Value Count by Feature After Cleaning')
plt.show()

In [ ]:
print(f'Samples Remaining: {len(data)}')

<h2 style="text-align: center;">Location and GPS Coordinates</h2>

In [ ]:
# Unique combos of state, location features then count, then sort

geo_combos = data.groupby(['state', 'location']).size()
geo_combos = geo_combos.reset_index(name='count')
geo_combos.sort_values(by='state', ascending=True)

# Discrepancy between MI and Mississippi combo.

In [ ]:
data[data['location'] == 'Mississippi']

# GPS coordinates => Burdett, Mississippi, most likely data entry error.

In [ ]:
# Correcting the state to MS for Mississippi
data.loc[data['location'] == 'Mississippi', 'state'] = 'MS'

In [ ]:
# plot of count by state 

state_count = data['state'].value_counts()

plt.figure(figsize=(10, 6))
plot3 = plt.bar(state_count.index, state_count.values)
plt.bar_label(plot3, label_type = 'edge')
plt.xlabel('State')
plt.ylabel('Count')
plt.title('Count of Samples by State')
plt.show()


In [ ]:
# select row if state is TX or KS
data[(data['state'] == 'TX') | (data['state'] == 'KS')]

# GPS Coordinates are in Kansas, not Texas, correct entries
data.loc[data['location'] == 'Texas', 'state'] = 'KS'
data.loc[data['location'] == 'Texas', 'location'] = 'Kansas'


In [ ]:
state_count_2 = data['state'].value_counts()

plt.figure(figsize=(10, 6))
plot4 = plt.bar(state_count_2.index, state_count_2.values)
plt.bar_label(plot4, label_type = 'edge')
plt.xlabel('State')
plt.ylabel('Count')
plt.title('Count of Samples by State Corrected')
plt.show()

<h2 style="text-align: center;">Plant ID and Tissue Features</h2>